# Machine Learning Assignment 2

Amiru Chandrasena & Kelvin Young

# Classifying Cells Bassed On Type

The Cells Dataset contains two components, one component is the CSV document containing the features about the cell and the cells corresponding image url. Secondly there is also a folder of images on cells in PNG format

# [1] Reading In The Data

The first step of data analysis is reading in the data from the CSV file, secondly the next step is exploring the data to look for errors and tredns in the dataset that may help with optimising the approach and the model.

In [ ]:
import zipfile
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
dataSet_cells = pd.read_csv('data/data_labels_mainData.csv')


## [1.1] Exploring The Data

Exploring the data involves firstly checking the head and the tail of the data to ensure that the CSV file has been compleately read in without errors. This includes checking the first couple of records, the last records and the features. Secondly the next step is to check for missing values to ensure that the model is not introduced to incomplete data and interfearing with the performance of the model.  

In [ ]:
dataSet_cells.head(10)

In [ ]:
dataSet_cells.tail(10)

In [ ]:
dataSet_cells.tail(10)

In [ ]:
null_counts = dataSet_cells.isnull().sum()

print(null_counts)

In [ ]:
dataSet_cells.describe()

In [ ]:
dataSet_cells.dtypes

## [1.2] Stripping Features 

It is importnat to drop unimportant fatures from the dataset for example 'InstanceID', 'patientID' ..etc are irrelevant to the training of the model. THe only important information that is used if the numeric column called 'cellType' that assigns a numeric value to the 'cellTypeName' attributes and the 'ImageName'. 

In [ ]:
import pandas as pd

# Drop the specified features
dataSet_cells.drop(["InstanceID", "patientID", "cellTypeName", "isCancerous"], axis=1, inplace=True)
dataSet_cells.head()

## [1.3] Splitting The Dataset Into Training, Validaiton and Testing Sets

Splitting the dataset into the three section allows for the effecive testing of the dataset. Allocating 80%(5937) of the dataset to the the training of the model allows for th emodel to have an increased accuracy, while allowin for the validation set to test the accuracy of the model without exposing it to the final test data leaving it unseen. The validation set allows for the base models accuracy to be gauged allowing infromation about the model to be taken such as if its overfitted or underfitted and what possible hyperparameters need to be changed. 

We then use a histogram to check the distribution of train set attributes is approximately equal to the distribution of test set attributes. This is important for the following reasons:

Generalisation: By ensuring that the data is a fair sample of the the whole dataset for bot hthe validaiton and the test set, you increase the likelihood that the model will be able to generalise well to unseen data. If the test set has a significantly different distribution, the model may struggle to make accurate predictions because it hasn't learned the relevant patterns or variations during training. Further there can be bias as the data in the validation set may skew one way and not be representing of the whole dataset. 

Ultimately this helps in building a robust and reliable model that can generalize well to unseen data.

In [ ]:
# We split the data into 80% of training Data, 10% Validation and 10% Testing
from sklearn.preprocessing import StandardScaler

train_data, test_data = train_test_split(dataSet_cells, test_size=0.2, random_state=42)
train_data, val_data = train_test_split(train_data, test_size=0.25, random_state=42)

print("Train data : {}, Val Data: {}, Test Data: {}".format(train_data.shape, val_data.shape, test_data.shape))
import matplotlib.pyplot as plt

# Plot histogram for train set
plt.figure(figsize=(10, 6))
plt.hist(train_data['cellType'], bins=range(4), alpha=0.5, label='Train')
plt.hist(val_data['cellType'], bins=range(4), alpha=0.5, label='Validation')
plt.hist(test_data['cellType'], bins=range(4), alpha=0.5, label='Test')
plt.xlabel('cellType')
plt.ylabel('Count')
plt.title('Distribution of cellType in Train, Validation, and Test sets')
plt.xticks(range(4))
plt.legend()
plt.show()


The graph above shows the spread of data and we see that the validation adn test set are very similar and representative of the whole dataset.

# [2] Model Development

## [2.1] Base Model

The base model begins with defining the dimensions of the image with (16*16*3) turinnig this into an array, secondly the hidden layer defines 256 neurons to start which will be tunned later to ensure that the model is working effectively. Thirdly as there are 4 different types of cells there will be 4 different output classes.

In [ ]:
INPUT_DIM = (16,16,3)
HIDDEN_LAYER_DIM = 256
OUTPUT_CLASSES = 4

## [2.2] Create a sequential API to build the model

There are three different types of models in tensorflow and the Sequential model was used for this NN as it is simplistic in the manner it works allowsing for models to be built layer by layer and the simplicity for it to be trained. 

The three layers were used in the model as first the image is flattend as mentioned in the previous cell, there will be no learning done in the layer and only image processing. THe second layer is responsible for the learning as it uses a sigmoid acitvation. THen the final layer is responcible for the output of the results. 

In [ ]:
import tensorflow as tf

model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=INPUT_DIM), ## Flatten the images into (16x16x3) array
    tf.keras.layers.Dense(HIDDEN_LAYER_DIM, activation='sigmoid'),
    tf.keras.layers.Dense(OUTPUT_CLASSES)
])
model.summary()

In [ ]:
tf.keras.utils.plot_model(model, show_shapes=True)

## [2.3] Compiling The Model

In [ ]:
model.compile(optimizer='SGD',
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=['categorical_accuracy'])

Change the feature title to a string 

In [ ]:
train_data['cellType'] = train_data['cellType'].astype('str')
val_data['cellType'] = val_data['cellType'].astype('str')

## [2.4] Image Generation 

Systematically load in the data 

Preprocessing: Preprocess your images as needed to make them suitable for training with a feedforward neural network. This may involve resizing the images to a fixed size and converting them to a suitable format (e.g., grayscale or RGB channels).


In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255, data_format='channels_last')
val_datagen = ImageDataGenerator(rescale=1./255, data_format='channels_last')

batch_size = 32

train_generator = train_datagen.flow_from_dataframe(
        dataframe=train_data,
        directory='patch_images',
        x_col="ImageName",
        y_col="cellType",
        target_size=(16, 16),
        batch_size=batch_size,
        class_mode='categorical')

validation_generator = val_datagen.flow_from_dataframe(
        dataframe=val_data,
        directory='patch_images',
        x_col="ImageName",
        y_col="cellType",
        target_size=(16, 16),
        batch_size=batch_size,
        class_mode='categorical')


## [2.5] Fitting & Training The Model

Now Training the model

In [ ]:
history = model.fit(train_generator, validation_data=validation_generator, epochs=50, verbose=0)


In [ ]:
import matplotlib.pyplot as plt

def plot_learning_curve(train_loss, val_loss, train_acc, val_acc, metric_name='Accuracy'):
    # Plotting the Loss Curve
    plt.figure(figsize=(12, 4))
    plt.subplot(1, 2, 1)
    plt.plot(train_loss, 'r-', label='Training Loss')
    plt.plot(val_loss, 'b-', label='Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.title('Loss Curve')
    plt.legend()

    # Plotting the Accuracy Curve
    plt.subplot(1, 2, 2)
    plt.plot(train_acc, 'r-', label='Training ' + metric_name)
    plt.plot(val_acc, 'b-', label='Validation ' + metric_name)
    plt.xlabel('Epochs')
    plt.ylabel(metric_name)
    plt.title('Accuracy Curve')
    plt.legend()

    plt.tight_layout()
    plt.show()

# Call the function to plot the learning curves
plot_learning_curve(history.history['loss'], history.history['val_loss'], 
                    history.history['categorical_accuracy'], history.history['val_categorical_accuracy'], 
                    metric_name='Accuracy')


On the other hand, the spiky behavior of the validation loss suggests that the model is not generalizing well to unseen data. It indicates that the model's performance on the validation set is inconsistent and fluctuates significantly during training. This could be a sign of overfitting, where the model is learning to fit the training data too closely and struggles to generalize to new examples

The spikier and increasing trend of the training accuracy compared to the first graph could be an indication that the model is memorizing the training data, leading to high accuracy on the training set but poor generalization to new examples.

If the model is over fitting:
Regularization (Lasso/Ridge penalty): Regularization techniques like L1 (Lasso) and L2 (Ridge) regularization can help control the complexity of the model by adding a penalty term to the loss function. This penalty discourages large weights and encourages the model to learn simpler and more generalizable representations. You can experiment with different regularization strengths to find the right balance between reducing overfitting and preserving model performance.

Dropout: Dropout is a regularization technique where randomly selected neurons are temporarily "dropped out" during training. This helps prevent the model from relying too heavily on specific neurons and encourages the network to learn more robust and generalized features. Dropout introduces some level of randomness and acts as an ensemble of multiple sub-networks during training, reducing overfitting.

Reduce the number of neurons or layers: Simplifying the model architecture by reducing the number of neurons or layers can help reduce overfitting. A complex model with a large number of parameters is more prone to overfitting, especially if the training data is limited. By reducing the model's capacity, you can encourage it to learn more generalized patterns and reduce the risk of overfitting.

plus use scaling 

# Tuned Hyperparameters

# Final Model Testing